# Switch Eager / Graph Execution

* [tf.executing_eagerly](https://www.tensorflow.org/api_docs/python/tf/executing_eagerly)

> Checks whether the current thread has eager execution enabled.

* [tf.config.run_functions_eagerly](https://www.tensorflow.org/api_docs/python/tf/config/run_functions_eagerly)

> Enables / disables eager execution of tf.functions.

## Graph Mode

Cannot use:

* for/while -> Use ```tf.while_loop``` or ```tf.py_function```.
* if -> Use ```tf.cond``` or ```tf.py_function```.
* print -> Use ```tf.print```
* assert
* tf.Tensor.shape attribute -> Use ```tf.shape()```
* tf.Tensor.ndim attribute

In [1]:
import tensorflow as tf

2023-11-16 10:44:55.566336: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2023-11-16 10:44:55.591872: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-11-16 10:44:55.591891: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-11-16 10:44:55.591907: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2023-11-16 10:44:55.596693: I tensorflow/core/platform/cpu_feature_g

In [2]:
@tf.function
def my_func(a):
    for _ in range(3):
        a += 1
        print(f"Python loop. a is {a}")
        
    return a

a_fn = tf.function(my_func)

# Graph Execution

In [2]:
tf.config.run_functions_eagerly(False)

In [4]:
# Should show False, but can get True
tf.executing_eagerly()

True

### TF graph evaluation runs the loop for the first time.

In [5]:
# you will see print 3 times in the loop. 
print(a_fn(tf.constant(2)))

Python loop. a is Tensor("add:0", shape=(), dtype=int32)
Python loop. a is Tensor("add_1:0", shape=(), dtype=int32)
Python loop. a is Tensor("add_2:0", shape=(), dtype=int32)
tf.Tensor(5, shape=(), dtype=int32)


2023-11-16 10:39:55.994304: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


### Second time, there will be no print in the loop/

In [8]:
print(a_fn(tf.constant(2)))

tf.Tensor(5, shape=(), dtype=int32)



## List comprehension

List comprehension itself works but note that the return value is **single Tensor packing all the values**.

In [3]:
@tf.function
def comprehension_func(n):
    x = tf.constant([
        value for value in range(n)  
    ])
    print(f"x is {x}")        
    return x

comprehension_fn = tf.function(comprehension_func)

In [4]:
comprehension_fn(5)

x is Tensor("Const:0", shape=(5,), dtype=int32)


2023-11-16 10:45:10.871854: W tensorflow/core/common_runtime/gpu/gpu_device.cc:2211] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...


<tf.Tensor: shape=(5,), dtype=int32, numpy=array([0, 1, 2, 3, 4], dtype=int32)>

In [6]:
comprehension_fn(3)

x is Tensor("Const:0", shape=(3,), dtype=int32)


<tf.Tensor: shape=(3,), dtype=int32, numpy=array([0, 1, 2], dtype=int32)>

# Confirm Eager Model

# Eager Execution

In [12]:
tf.config.run_functions_eagerly(True)

In [13]:
tf.executing_eagerly()

True

### You will see print in the loop 3 times again in eager mode.

In [15]:
print(a_fn(tf.constant(1)))

Python loop. a is 2
Python loop. a is 3
Python loop. a is 4
tf.Tensor(4, shape=(), dtype=int32)
